In [23]:
import pandas as pd
import numpy as np

In [24]:
df = pd.read_csv("../../data/train.csv")

In [25]:
df.drop(columns=['id'], inplace=True)
df["outcome"] = df["outcome"].replace({"died": 0, "lived": 1, "euthanized": 2})
df.fillna('empty', inplace=True)
df = pd.get_dummies(df)

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
X = df.drop(columns=["outcome"])
y = df["outcome"]

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y ,random_state=0, test_size=0.25, shuffle=True)

In [29]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [30]:
def train(model, _X, _y):
    model.fit(_X, _y)    
    return model

In [31]:
def measure(_model, _X, _y):
    
    result = _model.predict(_X)
    
    print(accuracy_score(result, _y))
    print()
    print(confusion_matrix(result, _y, labels=[1, 0, -1]))
    print()
    print(classification_report(result, _y, labels=[1, 0, -1]))

In [41]:
def submit(_model, df_test, _X_train):
    id = df_test['id'].to_list()

    df_test.drop(columns=['id'], inplace=True)
    df_test.fillna('empty', inplace=True)
    df_test = pd.get_dummies(df_test)

    df_test['peristalsis_distend_small'] = False
    df_test['nasogastric_reflux_slight'] = False
    df_test['rectal_exam_feces_serosanguious'] = False

    df_test.rename(columns={'pain_moderate': 'pain_slight'}, inplace=True)

    df_test = df_test[_X_train.columns.to_list()]

    result = _model.predict(df_test)

    df_submission = pd.DataFrame(data={'id': id ,'outcome': list(result)})
    df_submission["outcome"] = df_submission["outcome"].replace({0: "died", 1: "lived", 2: "euthanized"})
    df_submission.to_csv("submission.csv", index = False)

In [42]:
from sklearn.preprocessing import StandardScaler

In [34]:
from xgboost import XGBClassifier
df_test = pd.read_csv("../../data/test.csv")
df_test.head()
parameters = {
    "max_depth": list(np.arange(10, 100, 10)),
    "learning_rate": list(np.arange(0.01, 1.1, .1)),
    "subsample": list(np.arange(0.1, 1.1, 0.1))
}

ss = StandardScaler()

model = train(XGBClassifier(), ss.fit_transform(X_train), y_train)
submit(model, df_test, X_train, ss)

c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


In [35]:
# from xgboost import XGBClassifier
# from sklearn.model_selection import GridSearchCV
# df_test = pd.read_csv("../../data/test.csv")
# df_test.head()
# parameters = {
#     "max_depth": list(np.arange(10, 100, 10)),
#     "learning_rate": list(np.arange(0.01, 1.1, .1)),
#     "subsample": list(np.arange(0.1, 1.1, 0.1))
# }
# grid = GridSearchCV(estimator=XGBClassifier(), param_grid=parameters, cv=10, n_jobs=-1)
# model = train(grid, X_train, y_train)
# submit(model, df_test, X_train)

In [36]:
# model.best_params_

In [40]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
df_test = pd.read_csv("../../data/test.csv")
df_test.head()
parameters = {"objective": "multi:softprob", "tree_method": "hist","num_class": 3,"eta":.03}
grid = GridSearchCV(estimator=XGBClassifier(), param_grid=parameters, cv=10, n_jobs=-1)
model = train(grid, X_train, y_train)
submit(model, df_test, X_train)

TypeError: Parameter grid for parameter 'objective' needs to be a list or a numpy array, but got 'multi:softprob' (of type str) instead. Single values need to be wrapped in a list with one element.

In [43]:
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
df_test = pd.read_csv("../../data/test.csv")
df_test.head()
parameters = {"objective": "multi:softprob", "tree_method": "hist","num_class": 3,"eta": .03}
grid = XGBClassifier(**parameters)
model = train(grid, X_train, y_train)
submit(model, df_test, X_train)

c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
c:\GitHub\Health-Outcomes-of-Horses\venv\lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  retur